# Working With Models

## Register Existing Model

The following example registers a model from a local file.

In [ ]:
from azureml.core import Workspace
from azureml.core import Model
import sklearn

ws = Workspace.from_config()
model = Model.register(workspace = ws,
    model_name='mnb-cooking',
    tags={'kind': 'demo'},
    model_path='./output/mnb_model.pkl',
    model_framework = Model.Framework.SCIKITLEARN,
    model_framework_version = sklearn.__version__)

# This is how you could delete the model
# model.delete()

# Working With Deployments

The following code sample deploys the previously registered model to *Azure Container Instances*.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 2)
service = Model.deploy(ws, 'my-deployed-model', [model], deployment_config = deployment_config)

Here you see how we can remove a deployment:

In [ ]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice

ws = Workspace.from_config()
webserv = Webservice(ws, 'my-deployed-model')
webserv.delete()